In [3]:
from collections import deque
import numpy as np
import pandas as pd
from data_wrangling import df_possible_flights
from pathlib import Path


In [4]:
def distance_calc(lat1, lon1, lat2, lon2):
    earth_radius = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Compute differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula for distance calculation
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Earth radius in kilometers
    earth_radius = 6371.0
    
    # Calculate the distance
    distance = earth_radius * c
    
    return np.round(distance, 2)


In [5]:
def create_matrix(df, airports):
    num_airports = len(airports)

    # Create an adjacency matrix with zeros
    adjacency_matrix = np.zeros((num_airports, num_airports), dtype=float)

    # Fill the adjacency matrix based on the flights
    for index, row in df.iterrows():
        source_index = airports.index(row["Source Airport"])
        dest_index = airports.index(row["Destination Airport"])
        lat1, lon1 = row["Src Latitude"], row["Src Longitude"]
        lat2, lon2 = row["Dest Latitude"], row["Dest Longitude"]
        
        distance = distance_calc(lat1, lon1, lat2, lon2)
        adjacency_matrix[source_index][dest_index] = distance

    source = [airports.index('JFK'), airports.index('LGA')]
    sink = [airports.index('SFO')]

    return adjacency_matrix, source, sink


In [6]:
def ford_fulkerson(graph, sources, terminals):
    def residual_capacity(u, v):
        return graph[u][v] - flow[u][v]

    def distance_augment(path):
        total_distance = sum(graph[path[i]][path[i + 1]] for i in range(len(path) - 1))

        # update flow of the edges and reverse edges along the path
        for i in range(len(path) - 1):
            u, v = path[i], path[i + 1]
            flow[u][v] += total_distance  # update augmenting path
            flow[v][u] -= total_distance  # update flow on residual edges

        return total_distance


    def bfs():
        visited = [False] * len(graph) # similar to the color system, but using True/False instead
        queue = deque(sources) # getting the first source in the list
        for source in sources:
            visited[source] = True

        # setting the sources as parents initially
        parent = {source: None for source in sources}

        # while there is a source in the queue, find flights
        # that connect from source to terminal
        while queue:
            # getting the first element in the queue
            u = queue.popleft()
            # go through the airports that are connected to the parent
            for v in range(len(graph)):
                # if it hasn't been visited and it can  be augmented
                if not visited[v] and residual_capacity(u, v) > 0:
                    # add it to the queue
                    queue.append(v)
                    visited[v] = True
                    # set the u the parent of v
                    parent[v] = u

                    # if v is a terminal then we will create a path for the
                    # flight
                    if v in terminals:
                        path = []
                        # adding cities into the path
                        while v is not None:
                            path.insert(0, v)
                            v = parent[v]
                        return path

        return None

    # Initialize flow to zero
    flow = np.zeros_like(graph)

    # Ford-Fulkerson algorithm
    total_flow = 0
    while True:
        augmenting_path = bfs()
        if augmenting_path is None:
            break
        min_capacity = distance_augment(augmenting_path)
        total_flow += min_capacity
        print(f"Flight Route: {[airports[node] for node in augmenting_path]}, "
              f"total distance: {min_capacity} km")

    # Print the visited airports in the final path
    final_path = bfs()
    if final_path is not None:
        print("Final Path:", " --> ".join(airports[node] for node in final_path))

    print("Max Flow:", total_flow)


In [7]:
df = df_possible_flights
airports = sorted(set(df["Source Airport"]).union(set(df["Destination Airport"])))
adjacency_matrix, sources, sinks = create_matrix(df_possible_flights, airports)

adjacency_df = pd.DataFrame(adjacency_matrix, columns=airports, index=airports)

ford_fulkerson(adjacency_matrix, sources, sinks)

# csv_file_path = "/Users/yuhanburgess/Documents/GitHub/AGP2/csv_files/distance_matrix.csv"
# adjacency_df.to_csv(csv_file_path, index=True, header=True) 



Flight Route: ['JFK', 'SFO'], total distance: 4151.79 km
Flight Route: ['JFK', 'AMS', 'SFO'], total distance: 14633.04 km
Flight Route: ['JFK', 'ATL', 'SFO'], total distance: 4656.8099999999995 km
Flight Route: ['JFK', 'AUS', 'SFO'], total distance: 4861.47 km
Flight Route: ['JFK', 'BOS', 'SFO'], total distance: 4641.42 km
Flight Route: ['JFK', 'BUR', 'SFO'], total distance: 4482.98 km
Flight Route: ['JFK', 'BWI', 'SFO'], total distance: 4239.71 km
Flight Route: ['JFK', 'CDG', 'SFO'], total distance: 14795.859999999999 km
Flight Route: ['JFK', 'CLE', 'SFO'], total distance: 4152.16 km
Flight Route: ['JFK', 'CLT', 'SFO'], total distance: 4557.5 km
Flight Route: ['JFK', 'CPH', 'SFO'], total distance: 14995.82 km
Flight Route: ['JFK', 'CUN', 'SFO'], total distance: 6377.57 km
Flight Route: ['JFK', 'CVG', 'SFO'], total distance: 4214.79 km
Flight Route: ['JFK', 'DCA', 'SFO'], total distance: 4263.17 km
Flight Route: ['JFK', 'DEN', 'SFO'], total distance: 4162.55 km
Flight Route: ['JFK', 'D